 # Presence of Heart Disease
**This notebook will explore a heart disease database collected from the University of California, Irvine Machine Learning Repository. I want to compare various machine learning algorithms as well as tune the most promising models in order to maximize the accuracy of predicting the presence of heart disease. This has various applications in the medical field of detecting patients who are mostly likely to experience a heart disease in the near future given a set of features such as age, sex, and chest pains.** 


The dataset can be found here: [UCI Machine Learning Repository - Statlog (Heart)](http://archive.ics.uci.edu/ml/heart_diseases/statlog+(heart))

I will be running the following models:
1. Logistic Regression
2. Decision trees
3. K-Nearest Neighbors
4. Linear Discriminant Analysis
5. Gaussian Naive Bayes
6. Support Vector Machines

as well as have a 70/30 train/test split


In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

# Load dataset

In [ ]:
# Load heart_diseases
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.dat"
heart_disease = pd.read_fwf(url, header=None)
heart_disease.head()

# Data Cleaning

Column 3 has most of our features so we split the column. Then, I create dummy binary variables for chest pain levels, ecg levels, and thalamus levels since they are categorical variables.

In [ ]:
split_data =heart_disease[3].str.split('\s+', expand = True)
del heart_disease[3]
heart_disease = pd.concat([heart_disease, split_data], axis = 1, ignore_index = True)
heart_disease.head()

In [ ]:
# Store and convert categorical variables into floats
chest_pain  =heart_disease[2].astype('float64')
ecg = heart_disease[6].astype('float64')
thal = heart_disease[12].astype('float64')

# Binarize categorical variables
chest_pain_bin = pd.get_dummies(chest_pain)
ecg_bin = pd.get_dummies(ecg)
thal_bin = pd.get_dummies(thal)
response = heart_disease[13]

# Delete columns
del heart_disease[13]
del heart_disease[12]
del heart_disease[6]
del heart_disease[2]

In [ ]:
heart_disease.head()

In [ ]:
# Mix partitioned data
heart_disease = pd.concat([heart_disease, chest_pain_bin, ecg_bin, thal_bin, response], axis = 1, ignore_index = True)

In [ ]:
# Ensure quality control
heart_disease.head()

In [ ]:
heart_disease.columns

In [ ]:
heart_disease.shape

In [ ]:
# Look at datatypes
pd.set_option('display.max_rows',500)
heart_disease.dtypes

In [ ]:
# Convert data types into float64 for arithmetic operations
heart_disease.iloc[:, 0:20] = heart_disease.iloc[:, 0:20].astype('float64')
heart_disease.dtypes

In [ ]:
# Set precision to 3 decimal places
pd.set_option('precision', 3)
heart_disease.describe()

Recall that we have encoded binary dummy variables. So far we have:

| Index | Column   |
|------|-----------------|
|   0  | Age|
| 1 | Sex (1-Male, 0-Female)|
|2 | Resting Blood Pressure |
|3 | Serum Cholesterol in mg/dl |
| 4 | Fasting blood sugar | 
| 5 | Maximum heart rate achieved |
| 6 | Exercise induced angina | 
| 7 | Old peak ST depression induced by exercise relative to rest |
| 8 | Slope of the peak exercise ST segment |
| 9 | Number of major vessels (0-3) colored by flourosopy |
|10 | (BINARY) Chest Pain - 1 |
| 11 | (BINARY) Chest Pain - 2 |
| 12 | (BINARY) Chest Pain - 3 |
| 13| (BINARY) Chest Pain - 4 | 
| 14 | (BINARY) Resting Electrocardiographic results - 0 |
| 15 | (BINARY) Resting Electrocardiographic results - 1 |
| 16 | (BINARY) Resting Electrocardiographic results - 2 |
| 17 | (BINARY) Thalamus: Normal (3) |
| 18 | (BINARY) Thalamus: Fixed defect (6) |
| 19 | (BINARY) Thalamus: Reversable defect ( 7) |
| 20 | Absence (1) or presence (2) of heart disease |

# Histogram Plots

In [ ]:
## Create histograms
heart_disease.hist( column = [0,1,2,3,4,5,6,7,8,9],sharex = False, sharey = False, xlabelsize = 1, ylabelsize = 1)
plt.show()

We observe that age reasonably follows a normal distribution. Also, resting blood pressure at the time the data was taken seems to follow a right skewed normal distribution. The levels of exercised induced angina also follows a right skewed normal distribution. Slope of the peak exercise ST segment is reasonably a normal distribution. Note that we have a class imbalance in sex, fasting blood sugar, number of major vessels colored by flouroscopy, and slope of the peaks.

In [ ]:
# Histogram based on chest pain levels
plt.bar(range(1,5),chest_pain.value_counts().sort_index())
plt.show()

In [ ]:
ecg.value_counts().sort_index()

In [ ]:
# Barplot based on electrocardiogram levels (0,1,2)
plt.bar(range(1,4),ecg.value_counts().sort_index())
plt.show()

In [ ]:
# Barplot based on thalamus
plt.bar(range(1,4), thal.value_counts().sort_index())
plt.show()

# Density Plots

In [ ]:
# Create density plots on non-binary data
heart_disease.iloc[:, [0,2,3,5,7,9]].plot(kind='density', subplots=True, layout=(3,4), sharex=False, legend=False, fontsize=1)
plt.show()

# Correlation matrix
In order to prevent the confusion between the binary variables I have encoded, I have decided to leave them off in order to see some basic relationships in the data.

In [ ]:
# See correlation between non-binary data
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(heart_disease.iloc[:, [0,2,3,5,7,9]].corr(), vmin=-1, vmax=1, interpolation='none')
fig.colorbar(cax)
plt.show()

# Validation dataset
I decide to have train/test split of 70/30 in order to get some reasonable results.

In [ ]:
array = heart_disease.values
X = array[:,0:19].astype(float)
Y = array[:,20]
validation_size = 0.3
seed = 7
X_train, X_validation, Y_train, Y_validation = cross_validation.train_test_split(X, Y,test_size=validation_size, random_state=seed)

# Evaluate Algorithms
I also perform a 15-fold cross-validation and use the accuracy of classification as my performance metric.

In [ ]:
# Test options and evaluation metric
num_folds = 15
num_instances = len(X_train)
seed = 7
scoring_accuracy = 'accuracy'

In [ ]:
# Store models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

In [ ]:
# Run LR, LDA, KNN, CART, NB, SVM on data
results = []
names = []
for name, model in models:
    kfold = cross_validation.KFold(n = num_instances, n_folds = num_folds, random_state = seed)
    cv_results = cross_validation.cross_val_score(model, X_train, Y_train, cv = kfold, scoring = scoring_accuracy)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

Linear Discriminant Analysis (LDA) gives us a warning that certain predictors are collinear. This makes sense due to our binary encoding of the variables. For example, an increase in scoring a 1 on chest pain implies a decrease on chest pains of 2,3,4 and so it underestimates the effect of scoring a 1 on heart disease.

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# Standardize data due to distributions
Note that variables such as age carry the most weight in the classification process and so I decide to standardize the data points to the standard normal distribution.

In [ ]:
# Standardize the heart_disease
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR', LogisticRegression())])))
pipelines.append(('ScaledLDA', Pipeline([('Scaler', StandardScaler()),('LDA', LinearDiscriminantAnalysis())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsClassifier())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeClassifier())])))
pipelines.append(('ScaledNB', Pipeline([('Scaler', StandardScaler()),('NB', GaussianNB())])))
pipelines.append(('ScaledSVM', Pipeline([('Scaler', StandardScaler()),('SVM', SVC())])))
results = []
names = []
for name, model in pipelines:
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    cv_results = cross_validation.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring_accuracy)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# Fine tune k Nearest neighbors
By default, k-Nearest Neighbors runs 5 neighbors so I'd like to tune the parameter to see if we can achieve better accuracy. 

In [ ]:
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
neighbors = [1,3,5,7,9,11,13,15,17,19,21]

param_grid = dict(n_neighbors=neighbors)
model = KNeighborsClassifier()
kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring_accuracy, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
    

# Fine tuning Logistic Regression
Logistic Regression showed us the best promise and so by changing the hyperparameter, C, I hope to achieve better accuracy as we cross-validate.

In [ ]:
# Fine tune Logistic Regression
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
param_grid = {'C': [.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000] }
kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
grid = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, scoring=scoring_accuracy, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
    

# Fine tuning Support Vector Machine
SVM is one of those algorithms that can perform well if the kernel and hyperparameter, C, is tuned. Therefore, we perform a grid search to try various combinations.

In [ ]:
# Fine tune SVM parameter c as well as kernels
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
param_grid = dict(C=c_values, kernel=kernel_values)

kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
grid = GridSearchCV(estimator=SVC(), param_grid=param_grid, scoring=scoring_accuracy, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

# Ensemble Methods

In [ ]:
# Run emsemble methods
ensembles = []
ensembles.append(('AB', AdaBoostClassifier()))
ensembles.append(('GBM', GradientBoostingClassifier()))
ensembles.append(('RF', RandomForestClassifier()))
ensembles.append(('ET', ExtraTreesClassifier()))
results = []
names = []
for name, model in ensembles:
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    cv_results = cross_validation.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring_accuracy)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# Compare Algorithms 

In [ ]:
fig = plt.figure()
fig.suptitle('Ensemble Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# Performance on the Validation Set

In [ ]:
# Test LR model with C = .01
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = LogisticRegression(C=.01)
model.fit(rescaledX, Y_train)

# Test on cross validation set
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print("Logistic Regression")
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))


In [ ]:
# Test SVC with hyperparameter C= .03 and rbf 
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model2 = SVC(C=.5, kernel = "rbf")
model2.fit(rescaledX, Y_train)
predictions2 = model2.predict(rescaledValidationX)

print("Support Vector Machine")
print(accuracy_score(Y_validation, predictions2))
print(confusion_matrix(Y_validation, predictions2))
print(classification_report(Y_validation, predictions2))

In [ ]:
model3

In [ ]:
# Test kNN with 17 neighbors
model3 =  KNeighborsClassifier(n_neighbors = 17)
model3.fit(rescaledX, Y_train)
predictions3 = model3.predict(rescaledValidationX)

print( "k-Nearest Neighbors")
print(accuracy_score(Y_validation, predictions3))
print(confusion_matrix(Y_validation, predictions3))
print(classification_report(Y_validation, predictions3))


We conclude that by using logistic regression, we can achieve 84% accuracy in our model after fine tuning. 